In [2]:
import os
import time
import csv
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
from collections import deque

import gym
from gym3 import ToBaselinesVecEnv
from procgen import ProcgenEnv, ProcgenGym3Env
from procgen_tools import visualization, maze
from procgen_tools.imports import *
import matplotlib.pyplot as plt

In [37]:
# Load models
policy, hook = load_model('15', 15)

In [79]:
# Record behavior of model
import imageio
#temporarily choose one venv

# Load environments
venv_cheese = maze.create_venv(1, 0, 0, env_name='maze')
venv_redgem = maze.create_venv(1, 0, 0, env_name='maze_redgem_yellowstar')
venv_yellowstar = maze.create_venv(1, 0, 0, env_name='maze_yellowstar_redgem')

venvs = [venv_cheese, venv_redgem, venv_yellowstar]
venvs_names = ['cheese', 'redgem', 'yellowstar']

for i, venv in enumerate(venvs):

    frames = []
    observation = venv.reset()
    observation = torch.tensor(observation, dtype=torch.float32)  

    for j in range(100):
        frames.append(venv.render(mode='rgb_array')) #mode='rgb_array'
        action_probs, _ = policy(observation)  # replace with your model's prediction method
        action = action_probs.sample().item()
        action_np = np.array([action], dtype=np.int32)
        observation, reward, done, info = venv.step(action_np)
        observation = torch.tensor(observation, dtype=torch.float32)  

        if done:
            break

    print(f"Env: {venvs_names[i]}. Episode finished after {j+1} timesteps")

    imageio.mimwrite(f'recordings/output_{venvs_names[i]}.gif', frames, format='.gif')


Env: cheese. Episode finished after 67 timesteps
Env: redgem. Episode finished after 7 timesteps
Env: yellowstar. Episode finished after 100 timesteps
